# Yelp Data Mining Project
## CMPE - 255 Data Mining Fall 2017
### Group 6
- Dhrumil Shah
- Nishant Rathi
- Rashmi Sharma

## Notebook to Preprocess Data and create CSR

In this Jupyter Notebook, we are loading data from yelp data set sql dump. We have considered data using following constraints:
- Users who have written 50 or more reviews
- Their reviews
- Rating of each of the review

This data is grouped into 80% Training data and 20% Test data.
We have grouped data by user ID and then split it.

In [23]:
import random
import numpy as np
import pandas as pd

In [24]:
def test_train_split_per_groupby(review_data,groupby, testperc):
    uids=review_data[groupby].unique()
    full_size = len(uids)
    choose = (int)(((float)(testperc)/100) * full_size)
    indices = np.random.choice(len(uids), choose, replace=False)
    full_indices = [m for m in range(0,len(uids))]
    train_indices = set(full_indices)-set(indices)
    grouped =review_data.groupby([groupby])
    test_indices_final=[]  
    train_indices_final = []
    for g in grouped.groups:
        full_size = len(grouped.groups[g])
        choose = (int)(((float)(testperc)/100) * full_size)
        indices = np.random.choice(full_size, choose, replace=False)
        full_indices = [m for m in range(0,full_size)]
        train_indices = set(full_indices)-set(indices)
        train_indices = list(train_indices)
        test_indices_final.extend(list(grouped.groups[g][indices]))
        train_indices_final.extend(list(grouped.groups[g][train_indices]))
    saveGroupedData("data/test_"+str(groupby)+".dat",test_indices_final)
    saveGroupedData("data/train_"+str(groupby)+".dat",train_indices_final)

def saveGroupedData(filename, indices):
    output_file = open(filename, 'w')
    for i in indices:
        output_file.write(str(i)+"\n")
    output_file.close()    

In [26]:
review_data_all = pd.read_csv('data/reviews_no_text.dat',"\t")
num_samples=50000
review_data_all=review_data_all[:num_samples]
train_indices_user = test_train_split_per_groupby(review_data_all,'UID',20)

### Creating pickle files for user local model. i.e all per user reviews split into test and train

In [27]:
from util import *
user_id = ''
filename = 'pickle/'
user = load_pickle(filename+'userid.pickle')
ratings = load_pickle(filename+'rating.pickle')
business = load_pickle(filename+'business.pickle')
docs_csr = load_pickle(filename+'docs_csr.pickle')
docs_nmf = load_pickle(filename+'nmf.pickle')

unique_users = set(user)
unique_business = set(business)
doc_full = []
with open("data/test_UID.dat", "r") as fh:
    lines = fh.readlines()
with open("data/reviews_full.dat", "r") as fh:
    original_docs_lines = fh.readlines()  



user2 = []
ratings2 = []
business2 = []
#docs_csr2 = []
for i in lines:
    user2.append(user[int(i)])
    ratings2.append(ratings[int(i)])
    business2.append(business[int(i)])
    doc_full.append(original_docs_lines[int(i)].split('\t', 4)[3])


lines1 = map(int, lines)
filename = 'pickle/test/uid/'
save_pickle(user2, filename+'userid.pickle')
save_pickle(ratings2, filename+'rating.pickle')
save_pickle(business2, filename+'business.pickle')
save_pickle(docs_csr[lines1], filename+'docs_csr.pickle')
save_pickle(doc_full, filename+'docs_full.pickle')
save_pickle(docs_nmf[lines1], filename+'docs_nmf.pickle')

In [28]:
with open("data/train_UID.dat", "r") as fh:
    lines = fh.readlines()
with open("data/reviews_full.dat", "r") as fh:
    original_docs_lines = fh.readlines()  
doc_full = []

user2 = []
ratings2 = []
business2 = []
#docs_csr2 = []
for i in lines:
    user2.append(user[int(i)])
    ratings2.append(ratings[int(i)])
    business2.append(business[int(i)])
    doc_full.append(original_docs_lines[int(i)].split('\t', 4)[3])

lines1 = map(int, lines)
filename = 'pickle/train/uid/'
save_pickle(user2, filename+'userid.pickle')
save_pickle(ratings2, filename+'rating.pickle')
save_pickle(business2, filename+'business.pickle')
save_pickle(docs_csr[lines1], filename+'docs_csr.pickle')
save_pickle(doc_full, filename+'docs_full.pickle')
save_pickle(docs_nmf[lines1], filename+'docs_nmf.pickle')

### Creating Data for Business Local Model

In [29]:
filename = 'pickle/train/uid/'
user = load_pickle(filename+'userid.pickle')
ratings = load_pickle(filename+'rating.pickle')
business = load_pickle(filename+'business.pickle')
docs_csr = load_pickle(filename+'docs_csr.pickle')
unique_users = set(user)
unique_business = set(business)


datframeb={}
datframeb['BusinessId']=business
datframeb = pd.DataFrame(datframeb)
test_train_split_per_groupby(datframeb,'BusinessId',20)


In [31]:

lines=[]
with open("data/reviews_full.dat", "r") as fh:
    lines = fh.readlines()  
    
userid = []
rating = []
docs = []
business = []
i = 0
j = 0
error_line_num = []
error_lines = []
for line in lines:
    try:
        i = i + 1
        l = line.split('\t', 4)
        userid.append(l[0])
        business.append(l[1])
        rating.append(l[2])
        docs.append(l[3])
        #d = clean(l[3])
        #kmers = getKmers(d)
        #d.extend(kmers)
        #docs.append(d)
    except Exception as e:
        j = j + 1
        error_line_num.append(i)
        error_lines.append(line)

print 'Training Data: Number of lines processed: ' + str(i)
print 'Training Data: Length of userid array: ' + str(len(userid))
print 'Training Data: Length of rating array: ' + str(len(rating))
print 'Training Data: Length of docs array: ' + str(len(docs))
print 'Training Data: Length of business array: ' + str(len(business))
print 'Training Data: Number of exceptions encountered: ' + str(j)



from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
n_samples = 50000
n_features = 15000
n_components = 20
n_top_words = 20

data_samples = docs[:n_samples]

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))


# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)

tfidf_nmf = nmf.transform(tfidf)

print("done in %0.3fs." % (time() - t0))
print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")

vocab = np.array(tfidf_vectorizer.get_feature_names())

save_pickle(tfidf_nmf, filename+'nmf.pickle')
save_pickle(vocab, filename+'vocab.pickle')



Training Data: Number of lines processed: 317586
Training Data: Length of userid array: 317586
Training Data: Length of rating array: 317586
Training Data: Length of docs array: 317586
Training Data: Length of business array: 317586
Training Data: Number of exceptions encountered: 0
Extracting tf-idf features for NMF...
done in 0.275s.
Fitting the NMF model (generalized Kullback-Leibler divergence) with tf-idf features, n_samples=1000 and n_features=15000...
done in 3.203s.

Topics in NMF model (generalized Kullback-Leibler divergence):


In [32]:
with open("data/test_BusinessId.dat", "r") as fh:
    lines = fh.readlines()
with open("data/reviews_full.dat", "r") as fh:
    original_docs_lines = fh.readlines()  

doc_full = []

user2 = []
ratings2 = []
business2 = []
#docs_csr2 = []
for i in lines:
    user2.append(user[int(i)])
    ratings2.append(ratings[int(i)])
    business2.append(business[int(i)])
    doc_full.append(original_docs_lines[int(i)].split('\t', 4)[3])


lines1 = map(int, lines)
filename = 'pickle/test/businessid/'
save_pickle(user2, filename+'userid.pickle')
save_pickle(ratings2, filename+'rating.pickle')
save_pickle(business2, filename+'business.pickle')
save_pickle(docs_csr[lines1], filename+'docs_csr.pickle')
save_pickle(doc_full, filename+'docs_full.pickle')

In [34]:
with open("data/train_BusinessId.dat", "r") as fh:
    lines = fh.readlines()
with open("data/reviews_full.dat", "r") as fh:
    original_docs_lines = fh.readlines()  

doc_full = []


user2 = []
ratings2 = []
business2 = []
#docs_csr2 = []
for i in lines:
    user2.append(user[int(i)])
    ratings2.append(ratings[int(i)])
    business2.append(business[int(i)])
    doc_full.append(original_docs_lines[int(i)].split('\t', 4)[3])


lines1 = map(int, lines)
filename = 'pickle/train/businessid/'
save_pickle(user2, filename+'userid.pickle')
save_pickle(ratings2, filename+'rating.pickle')
save_pickle(business2, filename+'business.pickle')
save_pickle(docs_csr[lines1], filename+'docs_csr.pickle')
save_pickle(doc_full, filename+'docs_full.pickle')